In [1]:
import streamlit as st

# Streamlit UI
title = st.title('Aplicación para procesar texto (Observatorio)')

# Input fields
texto_main = st.text_area('Texto', height=250)
texto_referencias = st.text_area('Referencias', height=250)

boton_procesar = st.button('Procesar')

if boton_procesar:
    st.write('Texto Principal:', texto_main)
    st.write('Referencias:', texto_referencias)

2024-08-30 20:13:48.138 
  command:

    streamlit run c:\Users\SALVADOR\AppData\Local\Programs\Python\Python310\lib\site-packages\ipykernel_launcher.py [ARGUMENTS]
2024-08-30 20:13:48.139 Session state does not function when running a script without `streamlit run`


In [ ]:
#
streamlit run C:\Users\SALVADOR\OneDrive\CEPLAN\CeplanPythonCode\automatizacionV3.py
# Process button
if st.button('Process'):
    st.write("Processing...")
    
    # Extract references
    references = extract_references(text_references)
    st.write(f"Extracted {len(references)} references")
    
    # Replace brackets with links
    text_with_links = replace_brackets(text_main, references)
    st.write(f"Text with links length: {len(text_with_links)}")
    
    # Add paragraph tags
    final_text = add_paragraph_tags(text_with_links)
    st.write(f"Final text length: {len(final_text)}")
    
    # Split into panels
    panels = split_into_panels(final_text)
    st.write(f"Split into {len(panels)} panels")
    
    # Display panels
    for i, panel in enumerate(panels):
        st.subheader(f'Panel {i+1}')
        st.markdown(panel, unsafe_allow_html=True)

    st.write("Processing completed")

In [ ]:
import re
# Functions
def extract_references(text):
    pattern = re.compile(r'\[(\d+)\]\s+.*?(https?://[^\s\]]+)', re.DOTALL | re.IGNORECASE)
    references = {}
    for match in pattern.finditer(text):
        number = int(match.group(1))
        url = match.group(2)
        # Remove trailing dot if present
        if url.endswith('.'):
            url = url[:-1]
        references[number] = url
    return references

def replace_brackets(text, references):
    pattern = re.compile(r'\[([\d,\s]+)\]')
    def replacement(match):
        numbers = match.group(1).split(',')
        links = []
        for number in numbers:
            number = int(number.strip())
            if number in references:
                links.append(f'<a href="{references[number]}">[{number}]</a>')
            else:
                links.append(f'[{number}]')
        return ' '.join(links)
    return re.sub(pattern, replacement, text)

def add_paragraph_tags(text):
    lines = text.splitlines()
    result = []
    for i, line in enumerate(lines):
        if i == 0:
            result.append(f'<p>{line}')
        else:
            result.append(f'</p>\n<p>{line}')
    result.append('</p>')
    return '\n'.join(result)

def split_into_panels(text):
    # Find all "Figure #" or "Table #" at the start of a line
    pattern = re.compile(r'^<p>(Figure|Table) \d+', re.MULTILINE)
    matches = list(pattern.finditer(text))
    
    if not matches:
        return [text]
    
    panels = []
    panels.append(text[:matches[0].start()].strip())
    
    for i in range(len(matches) - 1):
        panels.append(text[matches[i].start():matches[i+1].start()].strip())
    
    panels.append(text[matches[-1].start():].strip())
    
    return panels